# Create a cube using PyVista


---

This notebook outlines different ways to generate a cube using PyVista.


Import required modules.


In [ ]:
import numpy as np
import pyvista as pv
from scipy.spatial import ConvexHull
import utils

---

## 1 - Generate a cube using PyVista `pv.Cube()` method


Generate and plot the cube.


In [ ]:
cube_mesh = pv.Cube()
cube_mesh.plot(show_edges=True)

---

## 2 - Generating vertices


The first step to making a mesh of a cube is generating vertices. The simplest approach is to generate 8 vertices (one for each corner), but we can also generate more vertices.


### 2.1 - Generating 8 vertices


Make arrays of $x$, $y$, $z$ values.


In [ ]:
x = np.linspace(0, 1, 2)
y = np.linspace(0, 1, 2)
z = np.linspace(0, 1, 2)

Combine the $x$, $y$, $z$ values into an array, `vertices`.


In [ ]:
x_v, y_v, z_v = np.meshgrid(x, y, z)
vertices_small = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

Create a PyVista mesh; this mesh only has vertices (i.e. it is a point cloud). To make visualizing the point cloud easier, we will perform a Delaunay 3D tessellation before plotting.


In [ ]:
cube_mesh_small = pv.PolyData(vertices_small)

pl = pv.Plotter()
pl.add_mesh(cube_mesh_small.delaunay_3d())
pl.add_mesh(
    cube_mesh_small.points,
    color="black",
    point_size=5,
    render_points_as_spheres=True,
)
pl.show()

### 2.2 - Generating a variable number of vertices


Make arrays of $x$, $y$, $z$ values.


In [ ]:
x = np.linspace(0, 1, 20)
y = np.linspace(0, 1, 20)
z = np.linspace(0, 1, 20)

Generate vertices for each plane.


In [ ]:
# x = 0 plane.
x_v, y_v, z_v = np.meshgrid(np.array([0]), y, z)
vertices_x_0 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# x = 1 plane.
x_v, y_v, z_v = np.meshgrid(np.array([1]), y, z)
vertices_x_1 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# y = 0 plane.
x_v, y_v, z_v = np.meshgrid(x, np.array([0]), z)
vertices_y_0 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# y = 1 plane.
x_v, y_v, z_v = np.meshgrid(x, np.array([1]), z)
vertices_y_1 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# z = 0 plane.
x_v, y_v, z_v = np.meshgrid(x, y, np.array([0]))
vertices_z_0 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# z = 1 plane.
x_v, y_v, z_v = np.meshgrid(x, y, np.array([1]))
vertices_z_1 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

Combine the coordinates into one array, `vertices`. Remove duplicate elements from `vertices`.


In [ ]:
num_vertices = vertices_z_1.shape[0]
vertices_large = np.zeros((num_vertices * 6, 3), dtype=float)
for i, arr in enumerate(
    [vertices_x_0, vertices_x_1, vertices_y_0, vertices_y_1, vertices_z_0, vertices_z_1]
):
    vertices_large[i * num_vertices : (i + 1) * num_vertices, :] = arr

vertices_large = np.unique(vertices_large, axis=0)

Create a PyVista mesh; this mesh only has vertices (i.e. it is a point cloud). To make visualizing the point cloud easier, we will perform a Delaunay 3D tessellation before plotting.


In [ ]:
cube_mesh_large = pv.PolyData(vertices_large)

pl = pv.Plotter()
pl.add_mesh(cube_mesh_large.delaunay_3d())
pl.add_mesh(
    cube_mesh_large.points,
    color="black",
    point_size=5,
    render_points_as_spheres=True,
)
pl.show()

---

## 3 - Tessellating a 3D point cloud


We will use the method developed in the previous section to generate vertices.


In [ ]:
x = np.linspace(0, 1, 20)
y = np.linspace(0, 1, 20)
z = np.linspace(0, 1, 20)

# x = 0 plane.
x_v, y_v, z_v = np.meshgrid(np.array([0]), y, z)
vertices_x_0 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# x = 1 plane.
x_v, y_v, z_v = np.meshgrid(np.array([1]), y, z)
vertices_x_1 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# y = 0 plane.
x_v, y_v, z_v = np.meshgrid(x, np.array([0]), z)
vertices_y_0 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# y = 1 plane.
x_v, y_v, z_v = np.meshgrid(x, np.array([1]), z)
vertices_y_1 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# z = 0 plane.
x_v, y_v, z_v = np.meshgrid(x, y, np.array([0]))
vertices_z_0 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# z = 1 plane.
x_v, y_v, z_v = np.meshgrid(x, y, np.array([1]))
vertices_z_1 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

num_vertices = vertices_z_1.shape[0]
vertices = np.zeros((num_vertices * 6, 3), dtype=float)
for i, arr in enumerate(
    [vertices_x_0, vertices_x_1, vertices_y_0, vertices_y_1, vertices_z_0, vertices_z_1]
):
    vertices[i * num_vertices : (i + 1) * num_vertices, :] = arr

vertices = np.unique(vertices, axis=0)
cube_mesh = pv.PolyData(vertices)

There are several algorithms we can use to tessellate the 3D point cloud; we will use PyVista's `delaunay_3d()` method and SciPy's `spatial.ConvexHull()` method.


Perform the tessellation using PyVista's `delaunay_3d()` method.


In [ ]:
cube_mesh_delaunay = pv.PolyData(cube_mesh.points).delaunay_3d()
cube_mesh_delaunay.plot(show_edges=True)

Generate a triangles array using SciPy's `spatial.ConvexHull` method. Convert the triangles array to the format expected by PyVista, and create a new mesh.


In [ ]:
faces = ConvexHull(cube_mesh.points).simplices
sphere_mesh_convex_hull = pv.PolyData(cube_mesh.points)
sphere_mesh_convex_hull.faces = utils.numpy_faces_to_pyvista(faces)
sphere_mesh_convex_hull.plot(show_edges=True)

---

## 4 - Tessellating a 2D point cloud


The previous methods outlined in section 3 followed the following procedure:

1. Generate an array of vertices.
2. Tessellate the 3D point cloud.

In this section, we will first generate a 2D plan of a cube. We will tessellate this point cloud using a 2D tessellation algorithm. We will then "fold" the plan to create a cube. The procedure reads as follows:

1. Generate a 2D point cloud of the plan of of a cube.
2. Tessellate the 2D point cloud.
3. "Fold" the plan to create a cube, while keeping the same tessellation.

The code in this section is implemented in the `utils.cube()` function.


### 4.1 - Make a 2D mesh


We will start by generating 6 square meshes, one for each face of the cube. We will tesselate each face individually using the `delaunay_2d()` method.


In [ ]:
# Generate arrays for x, y values
N = 10
L = 2
x = np.linspace(-L / 2, L / 2, N)
y = np.linspace(-L / 2, L / 2, N)
z = np.array([0])

# Make an array of vertices for a square.
x_v, y_v, z_v = np.meshgrid(x, y, z)
square_vertices_2d = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

In [ ]:
# Generate vertices and triangles for the x = -L/2 face.
vertices_x_min = np.zeros(square_vertices_2d.shape)
vertices_x_min[:, 0] = -L / 2
vertices_x_min[:, 1] = square_vertices_2d[:, 1]
vertices_x_min[:, 2] = square_vertices_2d[:, 0]
faces_x_min = utils.pyvista_faces_to_numpy(
    pv.PolyData(vertices_x_min).delaunay_2d().faces
)

# Generate vertices and triangles for the x = +L/2 face.
vertices_x_max = np.zeros(square_vertices_2d.shape)
vertices_x_max[:, 0] = +L / 2
vertices_x_max[:, 1] = square_vertices_2d[:, 1]
vertices_x_max[:, 2] = square_vertices_2d[:, 0]
faces_x_max = utils.pyvista_faces_to_numpy(
    pv.PolyData(vertices_x_max).delaunay_2d().faces
)

# Generate vertices and triangles for the y = -L/2 face.
vertices_y_min = np.zeros(square_vertices_2d.shape)
vertices_y_min[:, 0] = square_vertices_2d[:, 0]
vertices_y_min[:, 1] = -L / 2
vertices_y_min[:, 2] = square_vertices_2d[:, 1]
faces_y_min = utils.pyvista_faces_to_numpy(
    pv.PolyData(vertices_y_min).delaunay_2d().faces
)

# Generate vertices and triangles for the y = +L/2 face.
vertices_y_max = np.zeros(square_vertices_2d.shape)
vertices_y_max[:, 0] = square_vertices_2d[:, 0]
vertices_y_max[:, 1] = L / 2
vertices_y_max[:, 2] = square_vertices_2d[:, 1]
faces_y_max = utils.pyvista_faces_to_numpy(
    pv.PolyData(vertices_y_max).delaunay_2d().faces
)

# Generate vertices and triangles for the z = -L/2 face.
vertices_z_min = np.zeros(square_vertices_2d.shape)
vertices_z_min[:, 0] = square_vertices_2d[:, 1]
vertices_z_min[:, 1] = square_vertices_2d[:, 0]
vertices_z_min[:, 2] = -L / 2
faces_z_min = utils.pyvista_faces_to_numpy(
    pv.PolyData(vertices_z_min).delaunay_2d().faces
)

# Generate vertices and triangles for the z = +L/2 face.
vertices_z_max = np.zeros(square_vertices_2d.shape)
vertices_z_max[:, 0] = square_vertices_2d[:, 1]
vertices_z_max[:, 1] = square_vertices_2d[:, 0]
vertices_z_max[:, 2] = L / 2
faces_z_max = utils.pyvista_faces_to_numpy(
    pv.PolyData(vertices_z_max).delaunay_2d().faces
)

### 4.2 - Combine the 2D meshes to make a cube


We will combine the 6 square meshes to make a cube. Once we have created a 3D mesh, we will use the `pyvista.PolyData.clean()` method to remove duplicate vertices, and update the triangles array.


In [ ]:
# Create an array of vertices.
cube_vertices_cartesian = np.vstack(
    (
        vertices_x_min,
        vertices_x_max,
        vertices_y_min,
        vertices_y_max,
        vertices_z_min,
        vertices_z_max,
    )
)

# Create an array of faces.
num_vertices = square_vertices_2d.shape[0]
cube_faces = utils.numpy_faces_to_pyvista(
    np.vstack(
        (
            faces_x_min,
            faces_x_max + num_vertices,
            faces_y_min + 2 * num_vertices,
            faces_y_max + 3 * num_vertices,
            faces_z_min + 4 * num_vertices,
            faces_z_max + 5 * num_vertices,
        )
    )
)

# Create a 3D mesh.
cube_mesh_cartesian_custom = pv.PolyData(cube_vertices_cartesian, cube_faces)
cube_mesh_cartesian_custom.clean()

# Plot the mesh.
cube_mesh_cartesian_custom.plot(show_edges=True, color="lightblue")